In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Libraries imported.


In [3]:
!pip3 install lxml
!pip install lxml
#import lxml

## Part 1: Read table from Wiki

In [33]:
ca_1=pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')[0]
print(ca_1.shape)
ca_1.head()

(180, 3)


,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [38]:
#drop rows with Borough ='Not assigned'
ca_2=ca_1[ca_1['Borough'] != 'Not assigned'].reset_index(drop=True)
print(ca_2.shape)
ca_2.head()

(103, 3)


,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [97]:
# It seems wiki page has been changed that neighborhood(s) has been grouped under one postal code.
# If a cell has a borough but a neighborhood = 'Not assigned', then neighborhood = borough
# since we have dropped borough = 'Not assigned' in ca_2 table, we can just locate neighorhood = 'Not assigned' and replce it with borough

ca_2.loc[ca_2['Neighbourhood'] == 'Not assigned', 'Neighbourhood_replaced']=ca_2['Borough']
ca_2.loc[ca_2['Neighbourhood'] != 'Not assigned', 'Neighbourhood_replaced']=ca_2['Neighbourhood']
ca_2=ca_2.drop(['Neighbourhood'], axis=1)
ca_2.rename(columns={'Neighbourhood_replaced':'Neighborhood'}, inplace=True)
print(ca_2.shape)
ca_2.head()


(103, 5)


,Postal Code,Borough,Latitude,Longitude,Neighborhood
0,M3A,North York,43.7545,-79.3300,Parkwoods
1,M4A,North York,43.7276,-79.3148,Victoria Village
2,M5A,Downtown Toronto,43.6555,-79.3626,"Regent Park, Harbourfront"
3,M6A,North York,43.7223,-79.4504,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,43.6641,-79.3889,"Queen's Park, Ontario Provincial Government"


## Part 2: Get latitude and the longitude coordinates

In [20]:
!pip3 install pgeocode
!pip install pgeocode

     |████████████████████████████████| 14.5 MB 214 kB/s eta 0:00:01███████████████████████▋      | 11.6 MB 5.2 MB/s eta 0:00:01
     |████████████████████████████████| 9.7 MB 11.1 MB/s eta 0:00:01
     |████████████████████████████████| 510 kB 36.6 MB/s eta 0:00:01
  Using cached https://files.pythonhosted.org/packages/35/88/688b8550ceeed59a4b0be3f54d2ad8075ce708b5d5c0b0ec1a3abe58d4cb/pgeocode-0.3.0-py3-none-any.whl


In [98]:
import pgeocode # import geocoder

nomi=pgeocode.Nominatim('ca')
for i in range(len(ca_2)):
    ca_2.loc[i,'Latitude'] = nomi.query_postal_code('{}'. format(ca_2.loc[i,'Postal Code'])).latitude
    ca_2.loc[i,'Longitude'] = nomi.query_postal_code('{}'.format(ca_2.loc[i,'Postal Code'])).longitude
ca_2.head()

,Postal Code,Borough,Latitude,Longitude,Neighborhood
0,M3A,North York,43.7545,-79.3300,Parkwoods
1,M4A,North York,43.7276,-79.3148,Victoria Village
2,M5A,Downtown Toronto,43.6555,-79.3626,"Regent Park, Harbourfront"
3,M6A,North York,43.7223,-79.4504,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,43.6641,-79.3889,"Queen's Park, Ontario Provincial Government"


## Part 3 : Analysis

### To make it simple, filter the Boroughs contain 'Toronto'

In [99]:
# To make it simple, filter the Boroughs contain 'Toronto'
ca_3=ca_2[ca_2['Borough'].str.contains('Toronto')].reset_index(drop=True)
print(ca_3.shape)
ca_3.head()

(39, 5)


,Postal Code,Borough,Latitude,Longitude,Neighborhood
0,M5A,Downtown Toronto,43.6555,-79.3626,"Regent Park, Harbourfront"
1,M7A,Downtown Toronto,43.6641,-79.3889,"Queen's Park, Ontario Provincial Government"
2,M5B,Downtown Toronto,43.6572,-79.3783,"Garden District, Ryerson"
3,M5C,Downtown Toronto,43.6513,-79.3756,St. James Town
4,M4E,East Toronto,43.6784,-79.2941,The Beaches


In [60]:
# Set up Foursquare API
CLIENT_ID = 'GPEVTUHCEN21MHG1TEH3AGA25LIH2GOPGUK30E25S2YVDJ3X' # your Foursquare ID
CLIENT_SECRET = 'U0BC0AYBTY4GO5SD34X3SRP4JKSQ0FGVL451EMKU3CTWIV41' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value
radius = 500

### Borrowed function from the NYC project to get the venues in each neighborhood

In [61]:
# Borrowed function from the NYC project
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [63]:
toronto_venues = getNearbyVenues(names=ca_3['Neighborhood'], latitudes=ca_3['Latitude'], longitudes=ca_3['Longitude'])

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West, Forest Hill Road Park
High Park, The Junction South
North Toronto West, Lawrence Park
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
R

In [67]:
print(toronto_venues.shape)
toronto_venues.head()

(1537, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.6555,-79.3626,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.6555,-79.3626,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.6555,-79.3626,Figs Breakfast & Lunch,43.655675,-79.364503,Breakfast Spot
3,"Regent Park, Harbourfront",43.6555,-79.3626,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot
4,"Regent Park, Harbourfront",43.6555,-79.3626,Berkeley Church,43.655123,-79.365873,Event Space


In [126]:
# seems 'Roseloawn' is not in foursquare query result
toronto_venues[toronto_venues['Neighborhood']=='Roselawn']

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category


### check how many venues in each neighbourhood

In [69]:
#check how many venues in each neighbourhood
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,96,96,96,96,96,96
"Brockton, Parkdale Village, Exhibition Place",39,39,39,39,39,39
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",14,14,14,14,14,14
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",58,58,58,58,58,58
Central Bay Street,61,61,61,61,61,61
Christie,12,12,12,12,12,12
Church and Wellesley,81,81,81,81,81,81
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
Davisville,25,25,25,25,25,25


#### Analyze each neighborhood

In [88]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns =  list(toronto_onehot[toronto_onehot.columns[~toronto_onehot.columns.isin(['Neighborhood'])]].columns)
fixed_columns.insert(0,'Neighborhood')
toronto_onehot = toronto_onehot[fixed_columns]

print(toronto_onehot.shape)
toronto_onehot.head()

(1537, 219)


,Neighborhood,Accessories Store,Adult Boutique,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Stadium,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Bike Rental / Bike Share,Bistro,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chiropractor,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Cafeteria,College Gym,College Rec Center,College Theater,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hawaiian Restaurant,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hot Dog Joint,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Lawyer,Library,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Market,Martial Arts School,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Store,Music Venue,New American Restaurant,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Photography Studio,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Poutine Place,Pub,Ramen Restaurant,Record Shop,Residential Building (Apartment / Condo),Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Yoga Studio
0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

#### Group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [93]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Accessories Store,Adult Boutique,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Stadium,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Bike Rental / Bike Share,Bistro,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chiropractor,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Cafeteria,College Gym,College Rec Center,College Theater,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hawaiian Restaurant,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hot Dog Joint,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Lawyer,Library,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Market,Martial Arts School,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Store,Music Venue,New American Restaurant,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Photography Studio,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Poutine Place,Pub,Ramen Restaurant,Record Shop,Residential Building (Apartment / Condo),Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Yoga Studio
0,Berczy Park,0.000000,0.000000,0.010417,0.020833,0.000000,0.000000,0.000000,0.000000,0.000000,0.010417,0.041667,0.000000,0.000000,0.000000,0.010417,0.000000,0.031250,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.010417,0.000000,0.000000,0.000000,0.000000,0.000000,0.010417,0.041667,0.000000,0.000000,0.020833,0.000000,0.000000,0.010417,0.031250,0.114583,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.010417,0.000000,0.010417,0.000000,0.000000,0.000000,0.010417,0.000000,0.000000,0.000000,0.000000,0.020833,0.000000,0.000000,0.010417,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010417,0.000000,0.000000,0.000000,0.000000,0.000000,0.020833,0.010417,0.000000,0.010417,0.000000,0.000000,0.010417,0.010417,0.010417,0.010417,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.010417,0.000000,0.000000,0.00,0.000000,0.00,0.010417,0.010417,0.

#### Print each neighborhood along with the top 5 most common venues

In [74]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
                venue  freq
0         Coffee Shop  0.11
1                Café  0.04
2               Hotel  0.04
3  Seafood Restaurant  0.04
4              Bakery  0.04


----Brockton, Parkdale Village, Exhibition Place----
            venue  freq
0            Café  0.08
1     Coffee Shop  0.08
2       Gift Shop  0.05
3             Bar  0.05
4  Breakfast Spot  0.05


----Business reply mail Processing Centre, South Central Letter Processing Plant Toronto----
                 venue  freq
0           Restaurant  0.14
1          Coffee Shop  0.14
2          Yoga Studio  0.07
3                 Bank  0.07
4  Martial Arts School  0.07


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
                venue  freq
0  Italian Restaurant  0.07
1         Coffee Shop  0.07
2                Café  0.05
3                 Bar  0.05
4           Speakeasy  0.03


----Central Bay Street----
                venue  freq
0 

#### A function to sort the venues in descending order

In [89]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

#### Create the new dataframe and display the top 10 venues for each neighborhood

In [121]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Hotel,Seafood Restaurant,Café,Bakery,Cocktail Bar,Beer Bar,Restaurant,Japanese Restaurant,Deli / Bodega
1,"Brockton, Parkdale Village, Exhibition Place",Café,Coffee Shop,Thrift / Vintage Store,Breakfast Spot,Bar,Gift Shop,Accessories Store,Boutique,Brewery,Diner
2,"Business reply mail Processing Centre, South C...",Restaurant,Coffee Shop,Yoga Studio,Intersection,Martial Arts School,Breakfast Spot,Bookstore,Sushi Restaurant,Japanese Restaurant,Italian Restaurant
3,"CN Tower, King and Spadina, Railway Lands, Har...",Coffee Shop,Italian Restaurant,Café,Bar,Speakeasy,Bank,Grocery Store,Gym / Fitness Center,French Restaurant,Park
4,Central Bay Street,Coffee Shop,Italian Restaurant,Sandwich Place,Hotel,Bubble Tea Shop,Middle Eastern Restaurant,Café,Restaurant,Plaza,Juice Bar


#### Run K-means to cluster neighborhoods, k=5

In [122]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:37] 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 3, 0, 0, 4,
       0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0], dtype=int32)

#### Create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood

In [123]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = ca_3

# merge toronto_grouped with manhattan_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head(37) # check the last columns!


,Postal Code,Borough,Latitude,Longitude,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,43.6555,-79.3626,"Regent Park, Harbourfront",0.0,Coffee Shop,Breakfast Spot,Yoga Studio,Theater,Health Food Store,Distribution Center,Pub,Electronics Store,Restaurant,Event Space
1,M7A,Downtown Toronto,43.6641,-79.3889,"Queen's Park, Ontario Provincial Government",0.0,Italian Restaurant,Sushi Restaurant,Beer Bar,Japanese Restaurant,Bubble Tea Shop,Martial Arts School,Mexican Restaurant,Coffee Shop,College Cafeteria,Burrito Place
2,M5B,Downtown Toronto,43.6572,-79.3783,"Garden District, Ryerson",0.0,Coffee Shop,Clothing Store,Café,Hotel,Middle Eastern Restaurant,Cosmetics Shop,Japanese Restaurant,Theater,Bubble Tea Shop,Fast Food Restaurant
3,M5C,Downtown Toronto,43.6513,-79.3756,St. James Town,0.0,Coffee Shop,Café,Seafood Restaurant,American Restaurant,Cocktail Bar,Italian Restaurant,Hotel,Department Store,Moroccan Restaurant,Creperie
4,M4E,East Toronto,43.6784,-79.2941,The Beaches,0.0,Pub,Trail,Gastropub,Health Food Store,Coffee Shop,Asian Restaurant,Bakery,Falafel Restaurant,Ethiopian Restaurant,Event Space
5,M5E,Downtown Toronto,43.6456,-79.3754,Berczy Park,0.0,Coffee Shop,Hotel,Seafood Restaurant,Café,Bakery,Cocktail Bar,Beer Bar,Restaurant,Japanese Restaurant,Deli / Bodega
6,M5G,Downtown Toronto,43.6564,-79.3860,Central Bay Street,0.0,Coffee Shop,Italian Restaurant,Sandwich Place,Hotel,Bubble Tea Shop,Middle Eastern Restaurant,Café,Restaurant,Plaza,Juice Bar
7,M6G,Downtown Toronto,43.6683,-79.4205,Christie,0.0,Café,Grocery Store,Candy Store,Park,Playground,Coffee Shop,Bank,Baby Store,Farmers Market,Event Space
8,M5H,Downtown Toronto,43.6496,-79.3833,"Richmond, Adelaide, King",0.0,Café,Coffee Shop,Restaurant,Gym,Salad Place,Hotel,Steakhouse,Thai Restaurant,Asian Restaurant,American Restaurant
9,M6H,West Toronto,43.6655,-79.4378,"Dufferin, Dovercourt Village",0.0,Park,Bakery,Pet Store,Pharmacy,Pizza Place,Pool,Middle Eastern Restaurant,Smoke Shop,Café,Furniture / Home Store


In [128]:
# as found earlier, 'Roselawn' is not in the foursqure query result, so drop it as it's NA
# drop row(s) if 1st Most Common Venue is NA
toronto_merged=toronto_merged.dropna(subset=['1st Most Common Venue']).reset_index(drop=True)
toronto_merged.head(37)

,Postal Code,Borough,Latitude,Longitude,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,43.6555,-79.3626,"Regent Park, Harbourfront",0.0,Coffee Shop,Breakfast Spot,Yoga Studio,Theater,Health Food Store,Distribution Center,Pub,Electronics Store,Restaurant,Event Space
1,M7A,Downtown Toronto,43.6641,-79.3889,"Queen's Park, Ontario Provincial Government",0.0,Italian Restaurant,Sushi Restaurant,Beer Bar,Japanese Restaurant,Bubble Tea Shop,Martial Arts School,Mexican Restaurant,Coffee Shop,College Cafeteria,Burrito Place
2,M5B,Downtown Toronto,43.6572,-79.3783,"Garden District, Ryerson",0.0,Coffee Shop,Clothing Store,Café,Hotel,Middle Eastern Restaurant,Cosmetics Shop,Japanese Restaurant,Theater,Bubble Tea Shop,Fast Food Restaurant
3,M5C,Downtown Toronto,43.6513,-79.3756,St. James Town,0.0,Coffee Shop,Café,Seafood Restaurant,American Restaurant,Cocktail Bar,Italian Restaurant,Hotel,Department Store,Moroccan Restaurant,Creperie
4,M4E,East Toronto,43.6784,-79.2941,The Beaches,0.0,Pub,Trail,Gastropub,Health Food Store,Coffee Shop,Asian Restaurant,Bakery,Falafel Restaurant,Ethiopian Restaurant,Event Space
5,M5E,Downtown Toronto,43.6456,-79.3754,Berczy Park,0.0,Coffee Shop,Hotel,Seafood Restaurant,Café,Bakery,Cocktail Bar,Beer Bar,Restaurant,Japanese Restaurant,Deli / Bodega
6,M5G,Downtown Toronto,43.6564,-79.3860,Central Bay Street,0.0,Coffee Shop,Italian Restaurant,Sandwich Place,Hotel,Bubble Tea Shop,Middle Eastern Restaurant,Café,Restaurant,Plaza,Juice Bar
7,M6G,Downtown Toronto,43.6683,-79.4205,Christie,0.0,Café,Grocery Store,Candy Store,Park,Playground,Coffee Shop,Bank,Baby Store,Farmers Market,Event Space
8,M5H,Downtown Toronto,43.6496,-79.3833,"Richmond, Adelaide, King",0.0,Café,Coffee Shop,Restaurant,Gym,Salad Place,Hotel,Steakhouse,Thai Restaurant,Asian Restaurant,American Restaurant
9,M6H,West Toronto,43.6655,-79.4378,"Dufferin, Dovercourt Village",0.0,Park,Bakery,Pet Store,Pharmacy,Pizza Place,Pool,Middle Eastern Restaurant,Smoke Shop,Café,Furniture / Home Store


#### Visualize the resulting clusters

In [129]:
# create map
nomi=pgeocode.Nominatim('ca')
latitude = nomi.query_postal_code('M5A').latitude
longitude = nomi.query_postal_code('M5A').longitude
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

#### check results

In [130]:
#cluster 1
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,0.0,Coffee Shop,Breakfast Spot,Yoga Studio,Theater,Health Food Store,Distribution Center,Pub,Electronics Store,Restaurant,Event Space
1,Downtown Toronto,0.0,Italian Restaurant,Sushi Restaurant,Beer Bar,Japanese Restaurant,Bubble Tea Shop,Martial Arts School,Mexican Restaurant,Coffee Shop,College Cafeteria,Burrito Place
2,Downtown Toronto,0.0,Coffee Shop,Clothing Store,Café,Hotel,Middle Eastern Restaurant,Cosmetics Shop,Japanese Restaurant,Theater,Bubble Tea Shop,Fast Food Restaurant
3,Downtown Toronto,0.0,Coffee Shop,Café,Seafood Restaurant,American Restaurant,Cocktail Bar,Italian Restaurant,Hotel,Department Store,Moroccan Restaurant,Creperie
4,East Toronto,0.0,Pub,Trail,Gastropub,Health Food Store,Coffee Shop,Asian Restaurant,Bakery,Falafel Restaurant,Ethiopian Restaurant,Event Space
5,Downtown Toronto,0.0,Coffee Shop,Hotel,Seafood Restaurant,Café,Bakery,Cocktail Bar,Beer Bar,Restaurant,Japanese Restaurant,Deli / Bodega
6,Downtown Toronto,0.0,Coffee Shop,Italian Restaurant,Sandwich Place,Hotel,Bubble Tea Shop,Middle Eastern Restaurant,Café,Restaurant,Plaza,Juice Bar
7,Downtown Toronto,0.0,Café,Grocery Store,Candy Store,Park,Playground,Coffee Shop,Bank,Baby Store,Farmers Market,Event Space
8,Downtown Toronto,0.0,Café,Coffee Shop,Restaurant,Gym,Salad Place,Hotel,Steakhouse,Thai Restaurant,Asian Restaurant,American Restaurant
9,West Toronto,0.0,Park,Bakery,Pet Store,Pharmacy,Pizza Place,Pool,Middle Eastern Restaurant,Smoke Shop,Café,Furniture / Home Store


In [131]:
#cluster 2
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
21,West Toronto,1.0,Park,Residential Building (Apartment / Condo),Dive Bar,Yoga Studio,Eastern European Restaurant,Flower Shop,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market


In [132]:
#cluster 3
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
30,Central Toronto,2.0,Coffee Shop,Light Rail Station,Supermarket,Liquor Store,Yoga Studio,Flower Shop,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market


In [133]:
#cluster 4
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
18,Central Toronto,3.0,Photography Studio,Park,Yoga Studio,Eastern European Restaurant,Flower Shop,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Falafel Restaurant


In [134]:
#cluster 5
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
22,Central Toronto,4.0,Playground,Gym Pool,Park,Garden,Donut Shop,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Falafel Restaurant
32,Downtown Toronto,4.0,Playground,Candy Store,Park,Grocery Store,Yoga Studio,Eastern European Restaurant,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market
